<a href="https://colab.research.google.com/github/donlap/stat424/blob/main/8_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🛠️ Lab Setup
!pip install equinox -q

import jax
import jax.numpy as jnp
import jax.random as jr
import equinox as eqx
import optax
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from glob import glob
from PIL import Image
from ipywidgets import interact, FloatSlider, Layout

# Global Configuration
IMAGE_SIZE = 100
LATENT_DIM = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3

# Set random seed
key = jr.PRNGKey(42)

print("Libraries installed. configuration set to:")
print(f"- Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"- Latent Sliders: {LATENT_DIM}")

In [ ]:
# @title 🎨 Load Data from GitHub
REPO_URL = "https://github.com/donlap/stat424"
FOLDER_NAME = "faces"

def load_data_from_github(repo_url, folder_name=""):
    repo_name = repo_url.split("/")[-1].replace(".git", "")

    if os.path.exists(repo_name):
        shutil.rmtree(repo_name)

    ret = os.system(f"git clone {repo_url}")
    if ret != 0:
        print("❌ Error cloning. Check URL.")
        return np.array([])

    search_path = os.path.join(repo_name, folder_name, "**", "*.png")
    files = glob(search_path, recursive=True)
    print(f"Found {len(files)} PNG files. Processing...")

    images = []
    for f in files:
        try:
            img = Image.open(f).convert("RGB")
            img_arr = np.array(img) / 255.0
            images.append(img_arr)
        except:
            pass

    shutil.rmtree(repo_name)

    return np.array(images)

data = load_data_from_github(REPO_URL, FOLDER_NAME)
print(f"✅ Successfully loaded {len(data)} images of shape {data.shape[1:]}")

# Display samples
plt.figure(figsize=(10, 2))
for i in range(min(5, len(data))):
    plt.subplot(1, 5, i+1)
    plt.imshow(data[i])
    plt.axis('off')
plt.show()

In [ ]:
# @title 🧠 Define VAE (Customized for 100x100)

class Encoder(eqx.Module):
    layers: list

    def __init__(self, key):
        keys = jr.split(key, 3)
        self.layers = [
            eqx.nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1, key=keys[0]),
            eqx.nn.Lambda(jax.nn.relu),
            eqx.nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1, key=keys[1]),
            eqx.nn.Lambda(jax.nn.relu),
        ]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x.flatten()

class Decoder(eqx.Module):
    linear: eqx.nn.Linear
    layers: list

    def __init__(self, latent_dim, key):
        keys = jr.split(key, 3)
        self.linear = eqx.nn.Linear(latent_dim, 64 * 25 * 25, key=keys[0])

        self.layers = [
            eqx.nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1, key=keys[1]),
            eqx.nn.Lambda(jax.nn.relu),
            eqx.nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1, key=keys[2]),
            # code here # Output [0, 1]
        ]

    def __call__(self, z):
        x = self.linear(z)
        x = x.reshape((64, 25, 25))
        for layer in self.layers:
            x = layer(x)
        return x

class VAE(eqx.Module):
    encoder: Encoder
    decoder: Decoder
    fc_mu: eqx.nn.Linear
    fc_logvar: eqx.nn.Linear

    def __init__(self, latent_dim, key):
        k1, k2, k3, k4 = jr.split(key, 4)
        self.encoder = Encoder(k1)
        self.decoder = Decoder(latent_dim, k2)

        flat_size = 64 * 25 * 25
        # code here


    def __call__(self, x, key):
        encoded = self.encoder(x)
        # code here




        return self.decoder(z), mu, log_var

# Initialize
key, subkey = jr.split(key)
model = VAE(latent_dim=10, key=subkey)
print(f"Model initialized with 3 Latent Dimensions for 100x100 images.")

In [ ]:
# @title 🏋️‍♀️ Train
# Loss function
def loss_fn(model, x, key):
    recon, mu, log_var = model(x, key)
    # code here


    return # code here

loss_fn_batch = jax.vmap(loss_fn, in_axes=(None, 0, 0))

# Optimizer
optimizer = optax.adam(LEARNING_RATE)
opt_state = optimizer.init(eqx.filter(model, eqx.is_array))

@eqx.filter_jit
def make_step(model, opt_state, x, key):
    keys = jr.split(key, x.shape[0])
    loss_val, grads = eqx.filter_value_and_grad(
        lambda m, x, k: jnp.mean(loss_fn_batch(m, x, k))
    )(model, x, keys)
    updates, opt_state = optimizer.update(grads, opt_state, model)
    model = eqx.apply_updates(model, updates)
    return model, opt_state, loss_val

# Training
EPOCHS = 1000
losses = []

print("Starting training...")
for epoch in range(EPOCHS):
    key, subkey = jr.split(key)
    perm = jr.permutation(subkey, len(data))
    shuffled_data = data[perm]

    epoch_loss = 0
    steps = 0

    for i in range(0, len(data), BATCH_SIZE):
        batch = shuffled_data[i : i + BATCH_SIZE]
        # JAX expects (C, H, W), Data is (H, W, C)
        batch = jnp.transpose(batch, (0, 3, 1, 2))

        key, sk = jr.split(key)
        model, opt_state, loss = make_step(model, opt_state, batch, sk)
        epoch_loss += loss
        steps += 1

    avg_loss = epoch_loss / steps
    losses.append(avg_loss)
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}: Loss {avg_loss:.2f}")

plt.plot(losses)
plt.title("Training Loss")
plt.show()

In [ ]:
# @title 🎛️ The 3-Slider Explorer

def generate_face(z0, z1, z2, z3, z4, z5, z6, z7, z8, z9):
    # Construct the latent vector from the 3 sliders
    z = jnp.array([z0, z1, z2, z3, z4, z5, z6, z7, z8, z9])

    # Decode
    img = model.decoder(z)

    # Format for display (C, H, W) -> (H, W, C)
    img = jnp.transpose(img, (1, 2, 0))

    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Latent: [{z0:.1f}, {z1:.1f}, {z2:.1f}, {z3:.1f}, {z4:.1f}, {z5:.1f}, {z6:.1f}, {z7:.1f}, {z8:.1f}, {z9:.1f}]")
    plt.show()

# Define the 3 specific sliders
slider_layout = Layout(width='500px')
z0_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 1', layout=slider_layout)
z1_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 2', layout=slider_layout)
z2_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 3', layout=slider_layout)
z3_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 4', layout=slider_layout)
z4_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 5', layout=slider_layout)
z5_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 6', layout=slider_layout)
z6_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 7', layout=slider_layout)
z7_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 8', layout=slider_layout)
z8_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 9', layout=slider_layout)
z9_slider = FloatSlider(min=-3, max=3, step=0.1, value=0, description='Feature 10', layout=slider_layout)

print("Interact with the 3 latent dimensions:")
interact(generate_face, z0=z0_slider, z1=z1_slider, z2=z2_slider, z3=z3_slider, z4=z4_slider, z5=z5_slider, z6=z6_slider, z7=z7_slider, z8=z8_slider, z9=z9_slider);

In [ ]:
# @title 📊 Visualize Latent Distributions (Grid View)
import math

def visualize_latents(model, data, batch_size=64):
    @eqx.filter_jit
    def encode_batch(model, x):
        def encode_single(img):
            return model.fc_mu(model.encoder(img))
        return jax.vmap(encode_single)(x)

    print("Encoding all images to latent space...")
    mus = []

    data_transposed = jnp.transpose(data, (0, 3, 1, 2))

    for i in range(0, len(data), batch_size):
        batch = data_transposed[i : i + batch_size]
        batch_mu = encode_batch(model, batch)
        mus.append(batch_mu)

    latent_vectors = np.concatenate(mus, axis=0)

    num_latents = latent_vectors.shape[1]
    cols = 5
    rows = math.ceil(num_latents / cols)

    fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
    axes = axes.flatten()

    print(f"Plotting distributions for {num_latents} latent variables...")

    x_ref = np.linspace(-4, 4, 100)
    y_ref = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x_ref**2)

    for i in range(num_latents):
        ax = axes[i]

        ax.hist(latent_vectors[:, i], bins=30, density=True, alpha=0.6, color='skyblue', label='Data')

        ax.plot(x_ref, y_ref, 'r--', linewidth=2, label='Ideal Standard Normal')

        ax.set_title(f"Latent Dim {i}")
        ax.set_yticks([])
        if i == 0:
            ax.legend(fontsize=8)

    for i in range(num_latents, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()


visualize_latents(model, data)